# Analysing and Visualising # 


## 1. Preparing the analysing enviroment ##
### importing packages and data ###



In [1]:
import pandas as pd
import requests
import os as os
import json 
import tweepy
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("twitter_archive_master.csv",index_col=0,
               dtype = {'tweet_id': str,
                        'in_reply_to_status_id ': str,
                        'in_reply_to_user_id': str,
                        'retweeted_status_id':str,
                        'retweeted_status_user_id':str,
                        'in_reply_to_status_id':str
                       })

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066 entries, 0 to 2065
Data columns (total 30 columns):
tweet_id                      2066 non-null object
in_reply_to_status_id         23 non-null object
in_reply_to_user_id           23 non-null object
timestamp                     2066 non-null object
source                        2066 non-null object
text                          2066 non-null object
retweeted_status_id           75 non-null object
retweeted_status_user_id      75 non-null object
retweeted_status_timestamp    75 non-null object
expanded_urls                 2066 non-null object
name                          2066 non-null object
doggo                         2066 non-null int64
floofer                       2066 non-null int64
pupper                        2066 non-null int64
puppo                         2066 non-null int64
rating                        2066 non-null float64
none                          2066 non-null int64
favorite_count                2066 non-n

In [4]:
df["timestamp"]=pd.to_datetime(df.timestamp)
df["retweeted_status_timestamp"]=pd.to_datetime(df.retweeted_status_timestamp)

In [5]:
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/892420643...,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaT,https://twitter.com/dog_rates/status/892177421...,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891815181...,...,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891689557...,...,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaT,https://twitter.com/dog_rates/status/891327558...,...,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True


In [6]:
df.describe()

,doggo,floofer,pupper,puppo,rating,none,favorite_count,retweet_count,img_num,p1_conf,p2_conf,p3_conf
count,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2.066000e+03,2.066000e+03
mean,0.038722,0.003872,0.107454,0.011617,1.169595,0.844627,8332.854792,2772.818490,1.203291,0.594568,1.346716e-01,6.034151e-02
std,0.192979,0.062122,0.309765,0.107179,3.995623,0.362347,12562.080796,4830.478374,0.562172,0.271062,1.007233e-01,5.094272e-02
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1584.500000,591.000000,1.000000,0.364254,5.387868e-02,1.621080e-02
50%,0.000000,0.000000,0.000000,0.000000,1.100000,1.000000,3664.500000,1305.000000,1.000000,0.588030,1.184015e-01,4.939645e-02
75%,0.000000,0.000000,0.000000,0.000000,1.200000,1.000000,10408.500000,3199.500000,1.000000,0.843883,1.955693e-01,9.208967e-02
max,1.000000,1.000000,1.000000,1.000000,177.600000,1.000000,162830.000000,82862.000000,4.000000,1.000000,4.880140e-01,2.734190e-01


# Visualisation #
## scatter matrix#

we are using this type of plotting to get a fast over view and insigt about our data set if there is any patter or some further data cleaning to be done 

In [ ]:
sns.set(style="ticks")

sns.pairplot(df)

KeyError: 'species'